In [41]:
# Libraries
from pathlib import Path
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os

In [42]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("–disable-extensions")

In [43]:
# Wildfire Column headers
table_1_headers  = ['Event_id','Episode','Glide_Number','Countries','Population_Affected','Max_Wind_speed','MAX_Storm_Surge','Vulnerability','Event_summary']
table_2_headers  = ['Event_id','Episode','Alert','N','Date','Max_Winds','Population_in_tropical_storm','Population_in_Cat1_and_above','Countries']
table_3_headers  = ['Event_id','Episode','Alert','N','Date','Category','Max_Winds','Population_in_Cat1_and_above', 'Population_in_Tropical_Storm_and_above','Lat/Long', 'Countries']
table_4_headers  = ['Event_id','Episode','Countries']
table_5_headers  = ['Event_id','Episode','Region_Province','Country']
table_6_headers  = ['Event_id','Episode','Name','Region_Province','Country','City_class']
table_7_headers  = ['Event_id','Episode','Name','IATA_Code','Elevation_in_m','Usage','Runway_type','IFR','Runway_Length_in_ft']
table_8_headers  = ['Event_id','Episode','Name','LOCODE','Country']
table_9_headers  = ['Event_id','Episode','Reservoir','Dam_Name','River','Year']
table_10_headers = ['Event_id','Episode','Site','Type','Name','Country']
table_11_headers = ['Event_id','Episode','Alert','Date ','Name','Country','Storm_Surge_Height']


# Main function
def create_csv(html_file):
    # File and event details
    file_name = html_file.split('/')[-1].split('.')[0]
    event_id, episode, event_type = file_name.split('_')[0:3]
    print(event_id, episode, event_type)

    # Initialize the WebDriver
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(html_file)


    # Define table configurations (XPaths and column limits)
    table_configs = [
        ('//*[@id="alert_summary_left"]/table/tbody', 7),
        ('//*[@id="impactTimeline"]/table/tbody', 7),
        ('//*[@id="bulletinTimeline"]/table/tbody', 9),
        ('//*[@id="countries"]/table/tbody', 1),
        ('//*[@id="provinces"]/table/tbody', 2),
        ('//*[@id="cities"]/table/tbody', 4),
        ('//*[@id="airports"]/table/tbody', 7),
        ('//*[@id="ports"]/table/tbody', 3),
        ('//*[@id="dams"]/table/tbody', 4),
        ('//*[@id="nuclear_npp"]/table/tbody', 5),
        ('//*[@id="locations"]/table/tbody', 5)
    ]

    # Process each table
    tables_data = []
    for i, (xpath, col_limit) in enumerate(table_configs):
        table_data = extract_table_data(driver, event_id, episode, xpath, col_limit)
        tables_data.append(table_data)
    for row in tables_data:
        print(row)
    driver.quit()

    # Define CSV destination
    destination_folder = 'CSV'
    absolute_folder_path = Path(destination_folder).resolve()

    # Generate CSV files
    for i, table_data in enumerate(tables_data,start=1):
        csv_file_path = f'{absolute_folder_path}\{event_type}_{i}.csv'
        file_exists = os.path.exists(csv_file_path)
        header = eval(f"table_{i}_headers")
        print(i,header)
        print(table_data)
        df = pd.DataFrame(table_data,columns=header)
        df.head()
        df.to_csv(csv_file_path, mode='a' if file_exists else 'w', header=not file_exists, index=False)
        print(f"Data {'appended to' if file_exists else 'written to'} {csv_file_path}")



In [44]:

# Function to extract table data based on XPath and the number of columns required
def extract_table_data(driver, event_id, episode, xpath, col_limit):
    table_data = []
    if xpath == '//*[@id="alert_summary_left"]/table/tbody' or xpath == '//*[@id="tab_responsive"]/table/tbody' :
        try:
            temp_tbl=[]
            table = driver.find_element(By.XPATH, xpath)
            rows = table.find_elements(By.XPATH, './/tr')
            if xpath == '//*[@id="alert_summary_left"]/table/tbody':
                event_summary = '//*[@id="alert_summary_left"]/p'
            else :
                if xpath == '//*[@id="tab_responsive"]/table/tbody':
                    event_summary = '//*[@class="p_summary"][1]'
            event_summary_text = driver.find_element(By.XPATH, event_summary).text
            for row in rows:
                columns = row.find_elements(By.XPATH, './/td')
                row_data = [column.text for column in columns]
                if len(row_data)>=1:
                    temp_tbl.append(row_data)
            table_data = [[event_id, episode,*[temp_tbl[i][1] for i in range(1,col_limit)]
                        ,event_summary_text]]
        except NoSuchElementException:
            print(f"Event summary not founds or table with XPath '{xpath}' not found.")
        return table_data
    else:
        try:
            table = driver.find_element(By.XPATH, xpath)
            rows = table.find_elements(By.XPATH, './/tr')
            for row in rows:
                columns = row.find_elements(By.XPATH, './/td' if col_limit else './/th | .//td')
                row_data = []
                for col in columns:
                    cell_text = col.text.strip()

                    # If the cell is empty, look for an image
                    if not cell_text:
                        img_element = col.find_element(By.TAG_NAME, 'img') if col.find_elements(By.TAG_NAME, 'img') else None
                        if img_element:
                            # Try to get the title attribute
                            image_title = img_element.get_attribute('title')
                            if image_title:
                                row_data.append(image_title)  # Add the image title if present
                            else:
                                # Try to get the alt attribute if title is absent
                                alt_text = img_element.get_attribute('alt')
                                if alt_text:
                                    row_data.append(alt_text)  # Add the alt text if present
                                else:
                                    # If no title or alt, extract from the image URL
                                    image_url = img_element.get_attribute('src')
                                    image_code = image_url.split('/')[-1].split('.')[0]  # Extract "8p" from URL
                                    row_data.append(image_code)  # Add the extracted part from the URL
                        else:
                            row_data.append("")  # Leave it blank if no image is founD
                    else:
                        # If the cell contains text, just add it
                        row_data.append(cell_text) 

                if len(row_data) >= 1:
                    table_data.append([event_id, episode] + row_data[:col_limit if col_limit else len(row_data)])
        except NoSuchElementException:
            print(f"Table with XPath '{xpath}' not found.")
    return table_data

In [45]:
def list_filenames(folder_path):
    try:
        # Get a list of all files in the specified folder
        filenames = os.listdir(folder_path)
        
        return filenames
    
    except FileNotFoundError:
        print(f"Folder not found: {folder_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

def main():
    folder_path = "latest_htmls/Cyclones"
    absolute_folder_path = Path(folder_path).resolve()
    filenames = list_filenames(absolute_folder_path)

    for file in filenames:
        absolute_file_path = absolute_folder_path / file
        create_csv(f"file:///{absolute_file_path.as_posix()}")


In [46]:
if __name__ == "__main__":
    main()

1000667 18 Cyclones
[['1000667', '18', 'TC-2024-000127-CHN', 'Japan, Taiwan, China, Philippines', '24.7 million in Category 1 or higher', '231 km/h', '2 m (25 Jul 07:00 UTC)', 'Medium (China)', 'Tropical Cyclone GAEMI-24 can have a high humanitarian impact based on the maximum sustained wind speed,exposed population and vulnerability.']]
[['1000667', '18', 'Green', '1', '19 Jul 2024 18:00', '176', '12.6 million', '110 thousand', 'Japan'], ['1000667', '18', 'Green', '2', '20 Jul 2024 00:00', '194', '10.4 million', '60 thousand', 'Japan'], ['1000667', '18', 'Green', '3', '20 Jul 2024 06:00', '185', '49.5 million', '110 thousand', 'Japan'], ['1000667', '18', 'Green', '4', '20 Jul 2024 12:00', '185', '109.4 million', '110 thousand', 'Japan'], ['1000667', '18', 'Orange', '5', '20 Jul 2024 18:00', '185', '182.9 million', '19.9 million', 'Taiwan, China'], ['1000667', '18', 'Orange', '6', '21 Jul 2024 00:00', '185', '252.5 million', '13.3 million', 'Taiwan, Japan, China'], ['1000667', '18', 'O